In [12]:
# Module Importations
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

In [3]:
# Project Module Importations
from src.data import load_data
from src.data import split_data
from src.features import hyperparameter_opt

In [4]:
# Load dataset.
original_dataset = load_data.load_motor_data()

In [5]:
# Split data into training and test sets
training_set, test_set = split_data.split_train_test(original_dataset, 0.8)

In [6]:
# Drop profile id data column
training_set = training_set.drop("profile_id", axis = 1)
print(training_set.describe())

ambient        coolant            u_d            u_q  \
count  199614.000000  199614.000000  199614.000000  199614.000000   
mean       -0.005043       0.005308       0.003792      -0.005359   
std         0.996490       1.002488       0.998747       1.001464   
min        -5.239872      -1.270450      -1.654002      -1.852853   
25%        -0.601386      -1.037872      -0.831843      -0.924537   
50%         0.266671      -0.178200       0.267391      -0.098490   
75%         0.686838       0.673512       0.358587       0.848608   
max         2.954662       2.296845       2.273808       1.788773   

         motor_speed         torque            i_d            i_q  \
count  199614.000000  199614.000000  199614.000000  199614.000000   
mean       -0.004852      -0.001687       0.004184      -0.001570   
std         1.001954       0.998525       0.999021       0.998389   
min        -1.353747      -3.339106      -3.235659      -3.329749   
25%        -0.951892      -0.267419      -0.75

In [9]:
# Create rotor target
rotor_training_data = training_set.drop("pm", axis = 1)
rotor_label_data = training_set["pm"].copy()

In [10]:
# Linear Regression Model (Target - Rotor Temperature)
lin_reg = LinearRegression()
lin_reg.fit(rotor_training_data, rotor_label_data)

# Compute RMSE via cross validation
scores_mse = cross_val_score(lin_reg, rotor_training_data, rotor_label_data, scoring = "neg_mean_squared_error", cv = 5)
scores_rmse = np.sqrt(-scores_mse)

# Compute MAE via cross validation
scores_mae = cross_val_score(lin_reg, rotor_training_data, rotor_label_data, scoring = "neg_mean_absolute_error", cv = 5)
scores_mae = -1 * scores_mae

# Print results
model_name = "Linear Regression Model"
print(model_name, "RMSE Mean:", scores_rmse.mean())
print(model_name, "RMSE Standard Deviation:", scores_rmse.std())

print(model_name, "MAE Mean:", scores_mae.mean())
print(model_name, "MAE Standard Deviation:", scores_mae.std())

Linear Regression Model RMSE Mean: 0.4749361664649533
Linear Regression Model RMSE Standard Deviation: 0.0021400886517142672
Linear Regression Model MAE Mean: 0.3614280996261001
Linear Regression Model MAE Standard Deviation: 0.0016560246676062818


In [ ]:
# Decision Tree Regressor Model (Target - Rotor Temperature)
tree_regr = DecisionTreeRegressor()
tree_regr.fit(rotor_training_data, rotor_label_data)

# Compute RMSE via cross validation
scores_mse = cross_val_score(tree_regr, rotor_training_data, rotor_label_data, scoring = "neg_mean_squared_error", cv = 5)
scores_rmse = np.sqrt(-scores_mse)

# Compute MAE via cross validation
scores_mae = cross_val_score(tree_regr, rotor_training_data, rotor_label_data, scoring = "neg_mean_absolute_error", cv = 5)
scores_mae = -1 * scores_mae

# Print results
model_name = "Decision Tree Regressor"
print(model_name, "RMSE Mean:", scores_rmse.mean())
print(model_name, "RMSE Standard Deviation:", scores_rmse.std())

print(model_name, "MAE Mean:", scores_mae.mean())
print(model_name, "MAE Standard Deviation:", scores_mae.std())

In [ ]:
# Random Forest Regressor Model (Target - Rotor Temperature)
forest_regr = RandomForestRegressor()
forest_regr.fit(rotor_training_data, rotor_label_data)

# Compute RMSE via cross validation
scores_mse = cross_val_score(forest_regr, rotor_training_data, rotor_label_data, scoring = "neg_mean_squared_error", cv = 5)
scores_rmse = np.sqrt(-scores_mse)

# Compute MAE via cross validation
scores_mae = cross_val_score(forest_regr, rotor_training_data, rotor_label_data, scoring = "neg_mean_absolute_error", cv = 5)
scores_mae = -1 * scores_mae

# Print results
model_name = "Random Forest Regressor"
print(model_name, "RMSE Mean:", scores_rmse.mean())
print(model_name, "RMSE Standard Deviation:", scores_rmse.std())

print(model_name, "MAE Mean:", scores_mae.mean())
print(model_name, "MAE Standard Deviation:", scores_mae.std())

In [11]:
# Support Vector Regression - Linear (Target - Rotor Temperature)
svr_lin_regr = SVR(kernel='linear', C=100, gamma='auto')
svr_lin_regr.fit(rotor_training_data, rotor_label_data)

# Compute RMSE via cross validation
scores_mse = cross_val_score(svr_lin_regr, rotor_training_data, rotor_label_data, scoring = "neg_mean_squared_error", cv = 5)
scores_rmse = np.sqrt(-scores_mse)

# Compute MAE via cross validation
scores_mae = cross_val_score(svr_lin_regr, rotor_training_data, rotor_label_data, scoring = "neg_mean_absolute_error", cv = 5)
scores_mae = -1 * scores_mae

# Print results
model_name = "SVR - Linear"
print(model_name, "RMSE Mean:", scores_rmse.mean())
print(model_name, "RMSE Std:", scores_rmse.std())

print(model_name, "MAE Mean:", scores_mae.mean())
print(model_name, "MAE Std:", scores_mae.std())

NameError: name 'SVR' is not defined

In [8]:
# Random Forest Regressor - Optimised Hyperparameters
forest_regr = RandomForestRegressor()

best_params, best_est, best_score = hyperparameter_opt.perform_gridsearch("Random Forest Regressor [Opt]", forest_regr, rotor_training_data, rotor_label_data)

Random Forest Best Params: {'max_features': 6, 'n_estimators': 30}
Random Forest Best Estimator: RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features=6, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=30, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)
Random Forest Best Score: 0.07246792663747967
